In [1]:
import pennylane as qml
import torch
import numpy as np
from pennylane import numpy as qnp
from scipy.stats import unitary_group

import qnetvo

In [162]:
dev = qml.device("default.qubit.torch", wires=3)
input_state = torch.tensor(unitary_group.rvs(2)[:, 0], requires_grad=False)
rho_target = torch.outer(input_state, input_state.conj())

print(input_state)
print(rho_target)

dev2 = qml.device("default.qubit", wires=3)

tensor([ 0.6537-0.5764j, -0.4861+0.0644j], dtype=torch.complex128)
tensor([[ 0.7596+0.0000j, -0.3549+0.2381j],
        [-0.3549-0.2381j,  0.2404+0.0000j]], dtype=torch.complex128)


In [84]:
def rx_rz_rotation(phi, psi, wires):
    qml.RX(phi, wires=wires)
    qml.RZ(psi, wires=wires)

def pauli_xz(wires):
    qml.PauliZ(wires=wires)
    qml.PauliX(wires=wires)

In [181]:

@qml.qnode(dev, interface="torch")
def hardcoded_teleportation(settings):
    qml.QubitStateVector(input_state, wires=0)

    qml.Hadamard(wires=[1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[0,1])
    qml.Hadamard(wires=[0])

    m1 = qml.measure(0)
    m2 = qml.measure(1)

    # qml.cond(m1 == 1, qml.PauliZ)(wires=2)
    # qml.cond(m2 == 1, qml.PauliX)(wires=2)
    qml.cond((m1 == 1) & (m2 == 0), qml.PauliZ)(wires=2)
    qml.cond((m2 == 1) & (m1 == 0), qml.PauliX)(wires=2)
    qml.cond((m2 == 1) & (m1 == 1), pauli_xz)(wires=2)
    
    # Return state on final wire
    return qml.density_matrix(wires=2)




@qml.qnode(dev, interface="torch")
def teleport(settings, input_state):
    # Prepare input state
    qml.QubitStateVector(input_state, wires=0)

    # Prepare Bell state
    qml.ArbitraryStatePreparation(settings[0], wires=[1, 2])
    # qml.Hadamard(wires=[1])
    # qml.CNOT(wires=[1,2])

    # qml.CNOT(wires=[0,1])
    # qml.Hadamard(wires=[0])

    # # Apply gates
    qml.ArbitraryUnitary(settings[1], wires=[0, 1])

    # Measure first two wires
    m1 = qml.measure(0)
    m2 = qml.measure(1)
    
    # Condition final wire on results    
    # options = qml.apply_to_measurement(decision)(m1, m2)
    
    qml.cond((m1 == 0) & (m2 == 0), qml.Rot)(*settings[2][0], wires=2)
    qml.cond((m1 == 1) & (m2 == 0), qml.Rot)(*settings[2][1], wires=2)
    qml.cond((m2 == 1) & (m1 == 0), qml.Rot)(*settings[2][2], wires=2)
    qml.cond((m2 == 1) & (m1 == 1), qml.Rot)(*settings[2][3], wires=2)
    
    # Return state on final wire
    return qml.density_matrix(wires=2)


@qml.qnode(dev2)
def teleport2(settings, input_state):
    # Prepare input state
    qml.QubitStateVector(input_state, wires=0)

    # Prepare Bell state
    qml.ArbitraryStatePreparation(settings[0:6], wires=[1, 2])
    # qml.Hadamard(wires=[1])
    # qml.CNOT(wires=[1,2])

    # qml.CNOT(wires=[0,1])
    # qml.Hadamard(wires=[0])

    # # Apply gates
    qml.ArbitraryUnitary(settings[6:21], wires=[0, 1])

    # Measure first two wires
    m1 = qml.measure(0)
    m2 = qml.measure(1)
    
    # Condition final wire on results    
    # options = qml.apply_to_measurement(decision)(m1, m2)
    
    qml.cond((m1 == 0) & (m2 == 0), qml.Rot)(*settings[21:24], wires=2)
    qml.cond((m1 == 1) & (m2 == 0), qml.Rot)(*settings[24:27], wires=2)
    qml.cond((m2 == 1) & (m1 == 0), qml.Rot)(*settings[27:30], wires=2)
    qml.cond((m2 == 1) & (m1 == 1), qml.Rot)(*settings[30:33], wires=2)
    
    # Return state on final wire
    return qml.density_matrix(wires=2)

In [146]:

hardcoded_teleportation([])


tensor([[0.9553+0.0000j, 0.0765+0.1920j],
        [0.0765-0.1920j, 0.0447+0.0000j]], dtype=torch.complex128)

In [182]:
def cost(settings, input_state):
    rho = teleport(settings, input_state)
    rho_target = torch.outer(input_state, input_state.conj())
    return - qml.math.fidelity(rho, rho_target)

def cost2(settings, input_state):
    rho = teleport2(settings, input_state)
    rho_target = np.outer(input_state, input_state.conj())
    return - qml.math.fidelity(rho, rho_target)



input_states = [
    torch.tensor(unitary_group.rvs(2)[:, 0], requires_grad=False)
    for i in range(10)
]

np_input_states = [
    qnp.array(unitary_group.rvs(2)[:, 0], requires_grad=False)
    for i in range(10)
]

def true_cost(settings, input_states):
    cost_val = 0
    for input_state in input_states:
        cost_val += cost(settings, input_state)

    return cost_val / len(input_states)

def true_cost2(settings, input_states):
    cost_val = 0
    for input_state in input_states:
        cost_val += cost2(settings, input_state)

    return cost_val / len(input_states)



print("teleport : ", teleport(settings, input_states[0]))
print("cost : ", cost(settings, input_states[0]))
print("taget : ", torch.outer(input_states[0], input_states[0].conj()))

true_cost2(np_settings, np_input_states)



teleport :  tensor([[ 0.6730+0.0000j, -0.1008-0.2693j],
        [-0.1008+0.2693j,  0.3270+0.0000j]], dtype=torch.complex128,
       grad_fn=<SqueezeBackward0>)
cost :  tensor(-0.1711, dtype=torch.float64, grad_fn=<NegBackward0>)
taget :  tensor([[0.2171+0.0000j, 0.0541+0.4087j],
        [0.0541-0.4087j, 0.7829+0.0000j]], dtype=torch.complex128)


ValueError: Weights tensor must be of shape (6,); got (2,).

In [168]:
settings = [
    torch.tensor(np.random.random((6)), requires_grad=True),
    torch.tensor(np.random.random((15)), requires_grad=True),
    torch.tensor(np.random.random((4, 3)), requires_grad=True)
]

np_settings = qnp.random.rand()
# input_states = [
#     torch.tensor(unitary_group.rvs(2)[:, 0], requires_grad=False)
#     for i in range(10)
# ]
input_states = [
    torch.tensor([1,0] , requires_grad=False),
    torch.tensor([0,1], requires_grad=False),
    torch.tensor([1/np.sqrt(2),1/np.sqrt(2)] , requires_grad=False),
    torch.tensor([1/np.sqrt(2),-1/np.sqrt(2)] , requires_grad=False),
    torch.tensor([1/np.sqrt(2),1j/np.sqrt(2)] , requires_grad=False),
]


optimizer = torch.optim.SGD(settings, lr=1)

for i in range(251):
    
    optimizer.zero_grad()
    res = true_cost(settings, input_states)
    res.backward()
    optimizer.step()
    
    if i % 50 == 0:
        print(i, true_cost(settings, input_states))

0 tensor(-0.5453, dtype=torch.float64, grad_fn=<DivBackward0>)


KeyboardInterrupt: 

In [161]:
test_input_states = [
    torch.tensor(unitary_group.rvs(2)[:, 0], requires_grad=False)
    for i in range(50)
]


print("cost : ", true_cost(settings, test_input_states))
print("teleport state : ", teleport(settings, test_input_states[0]))
print("target ", torch.outer(test_input_states[0], test_input_states[0].conj()))

cost :  tensor(-1.0000, dtype=torch.float64, grad_fn=<DivBackward0>)
teleport state :  tensor([[0.2751+0.0000j, 0.4281+0.1272j],
        [0.4281-0.1272j, 0.7249+0.0000j]], dtype=torch.complex128,
       grad_fn=<SqueezeBackward0>)
target  tensor([[0.2751+0.0000j, 0.4281+0.1272j],
        [0.4281-0.1272j, 0.7249+0.0000j]], dtype=torch.complex128)


In [193]:
np_settings = qnp.random.rand(33, requires_grad=True)

np_input_states = [
    qnp.array([1,0] , requires_grad=False),
    qnp.array([0,1], requires_grad=False),
    qnp.array([1/np.sqrt(2),1/np.sqrt(2)] , requires_grad=False),
    qnp.array([1/np.sqrt(2),-1/np.sqrt(2)] , requires_grad=False),
    qnp.array([1/np.sqrt(2),1j/np.sqrt(2)] , requires_grad=False),
]

blah_cost = lambda settings: true_cost2(settings, np_input_states)

optimizer = qml.GradientDescentOptimizer(stepsize=1)

for i in range(251):

    np_settings, cost_val = optimizer.step_and_cost(blah_cost, np_settings)
    
    if i % 50 == 0:
        print(i, cost_val)

/Users/brian/opt/anaconda3/envs/qnetvo-dev/lib/python3.8/site-packages/autograd/numpy/numpy_wrapper.py:156: ComplexWarning: Casting complex values to real discards the imaginary part
  return A.astype(dtype, order, casting, subok, copy)
/Users/brian/opt/anaconda3/envs/qnetvo-dev/lib/python3.8/site-packages/autograd/numpy/numpy_vjps.py:99: RuntimeWarning: divide by zero encountered in power
  defvjp(anp.sqrt,    lambda ans, x : lambda g: g * 0.5 * x**-0.5)


0 -0.5810263392124135
50 -0.7222905181765544
100 -0.7256482466570291
150 -0.791781956955881
200 -0.9041128931488929
250 -0.9982939704561391


In [194]:
np_test_input_states = [
    qnp.array(unitary_group.rvs(2)[:, 0], requires_grad=False)
    for i in range(50)
]


print("cost : ", true_cost2(np_settings, np_test_input_states))
print("teleport state : ", teleport2(np_settings, np_test_input_states[0]))
print("target ", qnp.outer(np_test_input_states[0], np_test_input_states[0].conj()))

cost :  -0.9981013987557017
teleport state :  [[ 0.70280003+0.j         -0.28090135-0.35891963j]
 [-0.28090135+0.35891963j  0.29719997+0.j        ]]
target  [[ 0.70325375+0.j         -0.28183576-0.35952262j]
 [-0.28183576+0.35952262j  0.29674625+0.j        ]]


In [10]:
prep_nodes = [
    qnetvo.PrepareNode(1, [0], lambda settings, wires, cc_wires: qml.Hadamard(wires=wires), 0),
    qnetvo.PrepareNode(1, [1, 2], lambda settings, wires, cc_wires: qnetvo.ghz_state(settings, wires), 0)
]

def measure_circuit(settings, wires, cc_wires):
    qml.CNOT(wires[0:2])
    qml.Hadamard(wires=[0])

    b0 = qml.measure(wires[0])
    b1 = qml.measure(wires[1])

    return [b0, b1]

cc_measure_nodes = [
    qnetvo.CCMeasureNode(1, [0,1], [0,1], measure_circuit, 0)
]

def pauli_xz(wires):
    qml.PauliX(wires)
    qml.PauliZ(wires)

def cc_in_circuit(settings, wires, cc_wires):

    qml.cond((cc_wires[0] == 1) & (cc_wires[1] == 0), qml.PauliZ)(wires=2)
    qml.cond((cc_wires[0] == 0) & (cc_wires[1] == 1), qml.PauliX)(wires=2)
    qml.cond((cc_wires[0] == 1) & (cc_wires[1] == 1), pauli_xz)(wires=2)
    

meas_nodes = [
    qnetvo.MeasureNode(1, 2, [2], cc_in_circuit, 0, cc_wires_in=[0,1])
]

In [11]:
ansatz = qnetvo.NetworkAnsatz(prep_nodes, cc_measure_nodes, meas_nodes)

In [12]:
@qml.qnode(ansatz.dev)
def circ():
    ansatz.fn([])
    return qml.density_matrix(2)


In [13]:
circ()

tensor([[0.5+0.j, 0.5+0.j],
        [0.5+0.j, 0.5+0.j]], requires_grad=True)

In [15]:
opt_prep_nodes = [
    qnetvo.PrepareNode(1, [0], lambda settings, wires, cc_wires: qml.Hadamard(wires=wires), 0),
    qnetvo.PrepareNode(1, [1, 2], lambda settings, wires, cc_wires: qml.ArbitraryStatePreparation(settings, wires), 6)
]

def measure_circuit(settings, wires, cc_wires):
    qml.ArbitraryUnitary(settings, wires)

    b0 = qml.measure(wires[0])
    b1 = qml.measure(wires[1])

    return [b0, b1]

opt_cc_measure_nodes = [
    qnetvo.CCMeasureNode(1, [0,1], [0,1], measure_circuit, 15)
]

def ry_rz(settings, wires):
    qml.RY(settings[0], wires)
    qml.RZ(settings[1], wires)

def cc_in_circuit(settings, wires, cc_wires):

    qml.cond((cc_wires[0] == 0) & (cc_wires[1] == 1), ry_rz)(settings[0:2], wires=2)
    qml.cond((cc_wires[0] == 1) & (cc_wires[1] == 0), ry_rz)(settings[2:4], wires=2)
    qml.cond((cc_wires[0] == 0) & (cc_wires[1] == 1), ry_rz)(settings[4:6], wires=2)
    qml.cond((cc_wires[0] == 1) & (cc_wires[1] == 1), ry_rz)(settings[6:8], wires=2)
    

opt_meas_nodes = [
    qnetvo.MeasureNode(1, 2, [2], cc_in_circuit, 8, cc_wires_in=[0,1])
]

In [16]:
opt_ansatz = qnetvo.NetworkAnsatz(opt_prep_nodes, opt_cc_measure_nodes, opt_meas_nodes)

In [25]:
init_settings = opt_ansatz.rand_network_settings()

@qml.qnode(opt_ansatz.dev)
def teleport(settings):
    opt_ansatz.fn(settings)
    return qml.density_matrix(2)

def cost(*settings):
    rho = teleport(settings)
    rho_target = np.array([[1,1],[1,1]])/2
    return - qml.math.fidelity(rho, rho_target)

opt_dict = qnetvo.gradient_descent(cost, init_settings)


iteration :  0 , score :  0.5708701522852179
elapsed time :  0.059288978576660156
iteration :  25 , score :  0.9299854436339567
elapsed time :  0.044187068939208984
iteration :  50 , score :  0.980427584173603
elapsed time :  0.043660879135131836
iteration :  75 , score :  0.9901438699975584
elapsed time :  0.04353904724121094
iteration :  100 , score :  0.9936266432710954
elapsed time :  0.04364204406738281
iteration :  125 , score :  0.9954184495419954
elapsed time :  0.03859305381774902
